# Football Data Transformation


In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append("src")

In [ ]:
from spark_session import create_spark_session

spark = create_spark_session()

In [3]:
from data_loader import load_data

(matches_df, elo_df), load_time = load_data(spark)

We would first like to define the country in `matches_df` by joining Club in `elo_df` with HomeTeam _or_ AwayTeam in `matches_df`, but also based on the first letter in the League.

We also need to remember that clubs can be part of different leagues at different times.

As for betting odds, we will include 1) OddHome, OddDraw, and OddAway to establish baseline probabilities for match outcomes from a major bookmaker, 2) MaxHome, MaxDraw, and MaxAway to identify market inefficiencies and potential value bets across bookmakers, 3) Over25 and Under25 to understand expected match dynamics and goal scoring patterns, and 4) HandiSize, HandiHome, and HandiAway to capture team strength disparities in uneven matchups. These will provide Spark MLlib with diverse probability signals from betting markets, enabling the model to detect patterns between odds movements and match outcomes while potentially identifying profitable betting opportunities. So we'll drop just MaxOver25 and MaxUnder25.


In [4]:
from data_transformer import transform_matches_df
from utils import count_nulls


matches_transformed_df, transforming_time = transform_matches_df(matches_df, elo_df)

print(f"Number of rows in matches_transformed_df: {matches_transformed_df.count()}")
print("Sample rows in matches_transformed_df:")
matches_transformed_df.show(2)


print("\nMissing values in matches_transformed_df:")
count_nulls(matches_transformed_df).show()

Number of rows in matches_transformed_df: 135836
Sample rows in matches_transformed_df:


+-------+------+----------+---------+----------+-------+--------+------+------+--------------------+-------+-------+---------+---------+---------+---------+----------+----------+-------+-------+-------+-------+-------+-------+------+-------+---------+---------+---------+
|Country|League|      Date| HomeTeam|  AwayTeam|EloDiff|FTResult|FTHome|FTAway|                  ID|HomeElo|AwayElo|Form3Home|Form5Home|Form3Away|Form5Away|HomeTarget|AwayTarget|OddHome|OddDraw|OddAway|MaxHome|MaxDraw|MaxAway|Over25|Under25|HandiSize|HandiHome|HandiAway|
+-------+------+----------+---------+----------+-------+--------+------+------+--------------------+-------+-------+---------+---------+---------+---------+----------+----------+-------+-------+-------+-------+-------+-------+------+-------+---------+---------+---------+
|    FRA|    F1|2000-07-28|Marseille|    Troyes|  99.77|       H|   3.0|   1.0|2000-07-28_Marsei...|1686.34|1586.57|      0.0|      0.0|      0.0|      0.0|      NULL|      NULL|   1.6

+-------+------+----+--------+--------+-------+--------+------+------+---+-------+-------+---------+---------+---------+---------+----------+----------+-------+-------+-------+-------+-------+-------+------+-------+---------+---------+---------+
|Country|League|Date|HomeTeam|AwayTeam|EloDiff|FTResult|FTHome|FTAway| ID|HomeElo|AwayElo|Form3Home|Form5Home|Form3Away|Form5Away|HomeTarget|AwayTarget|OddHome|OddDraw|OddAway|MaxHome|MaxDraw|MaxAway|Over25|Under25|HandiSize|HandiHome|HandiAway|
+-------+------+----+--------+--------+-------+--------+------+------+---+-------+-------+---------+---------+---------+---------+----------+----------+-------+-------+-------+-------+-------+-------+------+-------+---------+---------+---------+
|      0|     0|   0|       0|       0|      0|       0|     0|     0|  0|      0|      0|        0|        0|        0|        0|     54609|     54609|   2356|   2356|   2356|  21162|  21162|  21162| 27407|  27408|    20709|    20856|    20880|
+-------+------+

Let's partition the matches by country and league and save the partitioned in Parquet format.


In [5]:
from data_loader import write_parquet_data

_, write_time = write_parquet_data(matches_transformed_df)

In [6]:
from data_loader import load_parquet_data

matches_parquet_df, parquet_read_time = load_parquet_data(spark)

In [7]:
# Compare reading times
print(f"CSV load time: {load_time:.2f} seconds")
print(f"Transforming time: {transforming_time:.2f} seconds")
print(f"Write time: {write_time:.2f} seconds")
print(f"Parquet read time: {parquet_read_time:.2f} seconds")
print(f"Speedup: {(load_time + write_time + transforming_time) / parquet_read_time:.2f}x")

CSV load time: 4.28 seconds
Transforming time: 0.50 seconds
Write time: 8.93 seconds
Parquet read time: 0.15 seconds
Speedup: 91.16x


In [8]:
import subprocess

from config import MATCHES_OUTPUT_DIR, MATCHES_PATH
from data_loader import get_data_path


def get_file_or_dir_size(path: str) -> str:
    try:
        result = subprocess.run(["du", "-sh", path], capture_output=True, text=True, check=True)
        return result.stdout.split()[0]
    except subprocess.SubprocessError:
        return "0M"


csv_size = get_file_or_dir_size(get_data_path(MATCHES_PATH))
parquet_size = get_file_or_dir_size(get_data_path(MATCHES_OUTPUT_DIR))

print(f"Actual CSV size: {csv_size}")
print(f"Actual Parquet size: {parquet_size}")
print(
    f"Compression ratio: ~{round(100 - (int(parquet_size.rstrip('M')) / int(csv_size.rstrip('M')) * 100))}% (including transformation)"
)

Actual CSV size: 37M
Actual Parquet size: 11M
Compression ratio: ~70% (including transformation)


In [11]:
import os

matches_output_dir = get_data_path(MATCHES_OUTPUT_DIR)

if os.path.exists(matches_output_dir):
    country_partitions = [
        f for f in os.listdir(matches_output_dir) if os.path.isdir(os.path.join(matches_output_dir, f))
    ]
    print(f"Country partitions found: {len(country_partitions)}")

    # Look at leagues within each country partition
    league_partitions = {}
    for country_part in country_partitions:
        country_dir = os.path.join(matches_output_dir, country_part)
        leagues = [f for f in os.listdir(country_dir) if os.path.isdir(os.path.join(country_dir, f))]
        league_partitions[country_part] = leagues

    # Show some example partitions
    print("\nExample partition structure:")
    for country, leagues in list(league_partitions.items())[:5]:
        print(f"{country}: {leagues[:5]}{'...' if len(leagues) > 5 else ''}")

    # Count total league partitions
    total_leagues = sum(len(leagues) for leagues in league_partitions.values())
    print(f"\nTotal country partitions: {len(country_partitions)}")
    print(f"Total league partitions: {total_leagues}")
else:
    print(f"Path {matches_output_dir} does not exist")

Country partitions found: 19

Example partition structure:
Country=TUR: ['League=T1']
Country=SCO: ['League=SC2', 'League=SC1', 'League=SC0']
Country=GRE: ['League=G1']
Country=ITA: ['League=I2', 'League=I1']
Country=SWE: ['League=SWE']

Total country partitions: 19
Total league partitions: 29


In [10]:
spark.stop()